## Transforming Feature Types

**Author**: Peter Simon

**Label**: Modeling Options

### Scope

The scope of this notebook is to provide instructions on how to transform feature types within DataRobot.

### Background

This is the procedure we are going to follow:

* Calculate the feature importance for each trained model
* Get the feature ranking for each trained model
* Get the ranking distribution for each feature across models
* Sort by mean rank and visualize

### Requirements

- Python version 3.7.3
-  DataRobot API version 2.19.0. 
Small adjustments might be needed depending on the Python version and DataRobot API version you are using.

Full documentation of the Python package can be found here: https://datarobot-public-api-client.readthedocs-hosted.com

It is assumed you already have a DataRobot <code>Project</code> object.

#### Set dictionary
Let's set up a dict with the variables we want to change. NOTE: if you want to do more than one type transformation on a single variable, you will need to do this via a dataframe instead. Variable types must be one of the following: {'categorical', 'categoricalInt', 'numeric', 'text'}

In [2]:
import datarobot as dr
VAR_TYPES = {'region'            : 'text',
             'reviews_seasonal'  : 'categoricalInt',
             'min_score'         : 'categorical',
             'max_score'         : 'an example of an illegal type',
             'reviews_department': 'text',
             'locality'          : 'text',
             }

# build a dict of the new feature names - we'll need it later
new_features = {}

#### Initiate transformation

In [ ]:
print('Starting variable type transformations.')

# now we'll run through the dict and do the transforms one-by-one
for feat, transf in VAR_TYPES.items():
    new_name = feat + ' (' + transf + ')'
    try:
        new_features[feat] = project.create_type_transform_feature(name=new_name,
                                                                   parent_name=feat,
                                                                   variable_type=transf).name
        print('Transformed', feat, 'to', new_name, 'successfully.')
    except dr.errors.ClientError as e:
        assert e.status_code == 422
        print(feat, 'transformation to', transf, 'failed.')

#### Create new feature list

In [ ]:
# let's make some feature lists which substitute the re-typed features for their parents
featurelists = project.get_featurelists()
new_featurelists = {}

print('Adding feature lists.')
for fl in featurelists:
    new_featlist_name = fl.name + ' (retyped)'
    new_featurelists[new_featlist_name] = project.create_featurelist(
        name=new_featlist_name,
        features=[new_features[f] if f in new_features.keys() else f for f in fl.features]
    ).id

#### Run autopilot

In [ ]:
# now run autopilot on the retyped Informative Features list
print('Starting autopilot.')
project.set_target(target=TARGET,
                   worker_count=-1,  # set to run with max. workers permitted
                   featurelist_id=new_featurelists['Informative Features (retyped)'],
                   )

# and invoke the browser, to carry on there
project.open_leaderboard_browser()